# Do injections and PE
We have two options for parallelizing:
1. Use SLURM (to submit jobs to a cluster), or
1. Use `multiprocessing` (to use the cores of the machine this notebook is running on).

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import sys
sys.path.append('../..')

import multiprocessing

from cogwheel.validation import inference
from cogwheel.validation import load_config

In [ ]:
config_filename = 'example/config.py'  # Edit as appropriate
config = load_config(config_filename)

In [ ]:
sample_inds = range(config.N_INJECTIONS)

rundirs = [inference.get_rundir(config, i_sample) for i_sample in sample_inds]

## Option 0: run a single one
(Probably worth doing this first, to check for bugs)

In [ ]:
inference.main(config_filename, rundirs[0])

## Option 1: SLURM

In [ ]:
for i_sample in sample_inds[:]:
    inference.submit_slurm(config_filename, i_sample, n_hours_limit=18, memory_per_task='12G')  # 32G

In [ ]:
!squeue 

## Option 2: `multiprocessing`

In [ ]:
with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
    pool.starmap(inference.main,
                 [(config_filename, rundir) for rundir in rundirs])